In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [ ]:
# Input Data Set #
Data ="BreastCancer"
JobNameAbbrev = "BC"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.01],
                       "Type": ["Classification"]}

### Include/exclude Random Forest Simulations

In [3]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.00],
#                        "rashomon_bound_adder": [0],
#                        "Type": ["Classification"]}
# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Create Parameter Vector

In [4]:
# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [5]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

In [6]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,BreastCancer,0,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,0BC_DPL0.01,BreastCancer/TreeFarms/Raw/0BC_DPL0.01.pkl
1,BreastCancer,1,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,1BC_DPL0.01,BreastCancer/TreeFarms/Raw/1BC_DPL0.01.pkl
2,BreastCancer,2,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,2BC_DPL0.01,BreastCancer/TreeFarms/Raw/2BC_DPL0.01.pkl
3,BreastCancer,3,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,3BC_DPL0.01,BreastCancer/TreeFarms/Raw/3BC_DPL0.01.pkl
4,BreastCancer,4,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,4BC_DPL0.01,BreastCancer/TreeFarms/Raw/4BC_DPL0.01.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,BreastCancer,95,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,95BC_DPL0.01,BreastCancer/TreeFarms/Raw/95BC_DPL0.01.pkl
96,BreastCancer,96,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,96BC_DPL0.01,BreastCancer/TreeFarms/Raw/96BC_DPL0.01.pkl
97,BreastCancer,97,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,97BC_DPL0.01,BreastCancer/TreeFarms/Raw/97BC_DPL0.01.pkl
98,BreastCancer,98,0.2,0.8,RandomForestClassificationFunction,TreeFarmsFunction,0,100,0.01,0.01,Classification,98BC_DPL0.01,BreastCancer/TreeFarms/Raw/98BC_DPL0.01.pkl
